# Saída estruturada

Nesta parte, quero explorar como fazer com que um LLM retorne uma saída estruturada.

LangChain oferece uma forma de acionar o LLM requisitando que a resposta siga uma certa estrutura.

## Configuração

Carregue variáveis de ambiente

In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

## Acesso ao LLM

Configure o LangChain para usar o LLM desejado.

In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(
    model="llama3.2", 
    temperature=0.0, 
    max_tokens=1000,
)

Esse é um texto de exemplo extraído da Wikipedia (https://pt.wikipedia.org/wiki/Campinas)

In [3]:
city_description = """Campinas é um município brasileiro no interior de São Paulo, Região Sudeste do Brasil. 
É um dos dezoito municípios que integram a Região Imediata de Campinas, que por sua vez é uma das onze
regiões imediatas que integram a Região Intermediária de Campinas.[7] Distante 99 km a noroeste de
São Paulo, capital estadual, ocupa uma área de 794,571 km². Em 2022, sua população foi estimada
pelo Instituto Brasileiro de Geografia e Estatística em 1 139 047 habitantes,[1] sendo, em 2021 o
terceiro município mais populoso de São Paulo (ficando atrás de Guarulhos e da capital) e o décimo
quarto de todo o país. Aparece em quinto lugar entre 100 municípios analisados pelo Índice das
Melhores e Maiores Cidades Brasileiras, o BCI100, elaborado pela Delta Economics & Finance com
base nos dados do Censo 2010 do IBGE e do Ideb.[8][9]
Campinas foi fundada em 14 de julho de 1774. Entre o final do século XVIII e o começo do século
XX, a cidade teve o café e a cana-de-açúcar como importantes atividades econômicas. Porém, desde a
década de 1930, a indústria e o comércio são as principais fontes de renda, sendo considerada um
polo industrial regional. Atualmente, é formada por seis distritos, além da sede, sendo, ainda,
subdividida em 14 administrações regionais, cinco regiões e vários bairros.[10]
Com o décimo segundo maior PIB entre os municípios do país, é a primeira cidade brasileira a se
tornar metrópole sem ser uma capital, exercendo significativa influência nacional.[11] Em 2011, foi
responsável por pelo menos 15% de toda a produção científica nacional, sendo o terceiro maior polo
de pesquisa e desenvolvimento brasileiro.[12] Tem também diversos atrativos turísticos, com valor
histórico, cultural ou científico, como museus, parques e teatros. A Orquestra Sinfônica Municipal de
Campinas, fundada em 1974, é considerada uma das principais do país."""

Essas são as informações desejadas sobre a cidade:

In [4]:
from pydantic import BaseModel, Field

class CityInformation(BaseModel):
    name: str = Field(description="Nome da cidade")
    area: str = Field(description="Área ocupada pela cidade", default="")
    foundation_date: str = Field(description="Data de fundação", default="")
    population: str = Field(description="População", default="")
    economy: str = Field(description="Atividade econômica atual", default="")

Aqui construímos o prompt a ser usado:

In [5]:
fields = ""
for key, field in CityInformation.model_fields.items():
    fields += f"- {key}: {field.description}\n"

prompt_template = """Analise o texto apresentado e extraia as seguintes informações:
{information_fields}
Se uma informação não for encontrada, deixe o campo vazio.
Escreva a resposta no formato JSON.

TEXTO:

{input_text}"""


prompt = prompt_template.format(information_fields=fields, input_text=city_description)

print(prompt)

Analise o texto apresentado e extraia as seguintes informações:
- name: Nome da cidade
- area: Área ocupada pela cidade
- foundation_date: Data de fundação
- population: População
- economy: Atividade econômica atual

Se uma informação não for encontrada, deixe o campo vazio.
Escreva a resposta no formato JSON.

TEXTO:

Campinas é um município brasileiro no interior de São Paulo, Região Sudeste do Brasil. 
É um dos dezoito municípios que integram a Região Imediata de Campinas, que por sua vez é uma das onze
regiões imediatas que integram a Região Intermediária de Campinas.[7] Distante 99 km a noroeste de
São Paulo, capital estadual, ocupa uma área de 794,571 km². Em 2022, sua população foi estimada
pelo Instituto Brasileiro de Geografia e Estatística em 1 139 047 habitantes,[1] sendo, em 2021 o
terceiro município mais populoso de São Paulo (ficando atrás de Guarulhos e da capital) e o décimo
quarto de todo o país. Aparece em quinto lugar entre 100 municípios analisados pelo Índice das


In [6]:
from rich.pretty import pprint

llm_struct_output = llm.with_structured_output(CityInformation)

response = llm_struct_output.invoke(
    prompt_template.format(information_fields=fields, input_text=city_description)
)

pprint(response)

CityInformation(
│   name='Campinas',
│   area='794,571 km²',
│   foundation_date='14 de julho de 1774',
│   population='1 139 047 habitantes (2022)',
│   economy='Indústria e comércio'
)

O que acontece se a informação não está presente?

In [7]:
other_description = """Campinas é um município brasileiro no interior de São Paulo, Região Sudeste do Brasil. 
É um dos dezoito municípios que integram a Região Imediata de Campinas, que por sua vez é uma das onze
regiões imediatas que integram a Região Intermediária de Campinas.[7] Distante 99 km a noroeste de
São Paulo, capital estadual, ocupa uma área de 794,571 km². Em 2022, sua população foi estimada
pelo Instituto Brasileiro de Geografia e Estatística em 1 139 047 habitantes,[1] sendo, em 2021 o
terceiro município mais populoso de São Paulo (ficando atrás de Guarulhos e da capital) e o décimo
quarto de todo o país. Aparece em quinto lugar entre 100 municípios analisados pelo Índice das
Melhores e Maiores Cidades Brasileiras, o BCI100, elaborado pela Delta Economics & Finance com
base nos dados do Censo 2010 do IBGE e do Ideb."""

response = llm_struct_output.invoke(
    prompt_template.format(information_fields=fields, input_text=other_description)
)

pprint(response)

CityInformation(
│   name='Campinas',
│   area='794,571 km²',
│   foundation_date='',
│   population='1 139 047 habitantes',
│   economy=''
)

In [8]:
other_description = """Campinas foi fundada em 14 de julho de 1774. Entre o final do século XVIII e o começo do século
XX, a cidade teve o café e a cana-de-açúcar como importantes atividades econômicas. Porém, desde a
década de 1930, a indústria e o comércio são as principais fontes de renda, sendo considerada um
polo industrial regional. Atualmente, é formada por seis distritos, além da sede, sendo, ainda,
subdividida em 14 administrações regionais, cinco regiões e vários bairros.[10]
Com o décimo segundo maior PIB entre os municípios do país, é a primeira cidade brasileira a se
tornar metrópole sem ser uma capital, exercendo significativa influência nacional.[11] Em 2011, foi
responsável por pelo menos 15% de toda a produção científica nacional, sendo o terceiro maior polo
de pesquisa e desenvolvimento brasileiro.[12] Tem também diversos atrativos turísticos, com valor
histórico, cultural ou científico, como museus, parques e teatros. A Orquestra Sinfônica Municipal de
Campinas, fundada em 1974, é considerada uma das principais do país."""

response = llm_struct_output.invoke(
    prompt_template.format(information_fields=fields, input_text=other_description)
)

pprint(response)

CityInformation(
│   name='Campinas',
│   area='',
│   foundation_date='14 de julho de 1774',
│   population='',
│   economy='Indústria e comércio'
)